<a href="https://colab.research.google.com/github/prof-Thiago-Nunes/Projetos-data-science-/blob/main/Geocoding_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Business Intelligence com GCP e Python*
*by Professor Thiago Nunes*

---

*Introdução*
O Google Cloud Platform (GCP) é uma plataforma de computação em nuvem fornecida pelo Google que permite aos usuários criar e executar aplicativos e serviços em uma infraestrutura de nuvem gerenciada. A plataforma oferece um conjunto de serviços de computação, armazenamento, banco de dados, análise, inteligência artificial e outros recursos que podem ser usados para construir, implantar e dimensionar aplicativos em nuvem.

Alguns dos serviços oferecidos pelo Google Cloud Platform incluem o Google Compute Engine para máquinas virtuais, o Google Kubernetes Engine para orquestração de contêineres, o Google Cloud Storage para armazenamento em nuvem, o BigQuery para análise de dados em larga escala, o Cloud AI Platform para aprendizado de máquina e o Cloud Functions para computação sem servidor.

<img src="https://nobug.com.br/wp-content/uploads/2023/02/gcp-google.jpg">

O GCP é usado por empresas e desenvolvedores para executar aplicativos e serviços em nuvem de forma escalável e segura, com alta disponibilidade e desempenho.

Neste projeto utilizarei a API do GCP (DocumentAI) para extrair dados de uma planilha de Excell e um pdf escaneado (a pior situação possível), são dados de clientes de uma loja contendo algumas informações como por exemplo o endereço de cada cliente. Logo após, será feito o tratamento dos dados e construiremos um modelo que verifica a distância de cada cliente para o local de um possível evento realizado pela empresa, para fazer isto será necessário utilizar a API do Google Maps.

## 1. ***📋 CHECKLIST* DO PROJETO**
✅ Transformar o pdf da planilha escaneada (`enderecos.pdf`) contendo os endereços dos clientes, em um DataFrame.
<br>✅ Obter as coordenadas (Latitude/Longitude) a partir dos endereços dos clientes.
<br>✅ Unificar os endereços com os dados pessoais dos clientes (`clientes_dados.csv`), a partir da coluna `id_cliente`.
<br>✅ Verificar a distância dos clientes até o local do evento, chamado "Estádio da Fonte Luminosa" de Araraquara.

<br>

Como será utilizado o Google Colab, é necessário criar uma pasta para os arquivos **dentro do seu Google Drive pessoal**.

Também usarei a autenticação do Document AI por meio das chaves que foram geradas, é importante a criação de uma chave de altenticação no GCP para o script rodar.


## 2. Instalação das bibliotecas e autenticação


In [63]:
# instalar produtos do Google Cloud
!pip install --upgrade google-cloud-documentai
!pip install googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
# Importar as bibliotecas necessárias
from google.oauth2 import service_account
from google.colab import drive
from typing import List, Sequence
import pandas as pd
from google.api_core.client_options import ClientOptions
from google.cloud import documentai_v1 as documentai
import json
import googlemaps
from datetime import datetime
import matplotlib.pyplot as plt

## 3. Configurando os arquivos no Drive

In [65]:
# Mount Google Drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [66]:
# Caminho até a sua pasta de projetos
PROJECT_PATH = "/content/drive/MyDrive/dados_empresa"

In [67]:
# Indicar o caminho até o arquivo JSON para autenticação
key_path = 'coloque sua chave no formato JSON' 


In [ ]:
# Inserir uma variável no ambiente
%env GOOGLE_APPLICATION_CREDENTIALS=$key_path

O que a linha `%env GOOGLE_APPLICATION_CREDENTIALS=$key_path` fará é criar uma variável no ambiente do servidor, com referência ao arquivo *json* que contém os dados para o Colab autenticar junto aos produtos do seu projeto no Google Cloud.

Obviamente estamos em um ambiente didático, mas fica o aviso: nunca compartilhe esse arquivo. Da mesma maneira, quando estiver lidando com *tokens* diretamente no *notebook*, tome cuidado para não publicar ou subir o arquivo em um repositório público.

Qualquer pessoa que detenha essa informação, irá ter acesso aos produtos ativados - ou seja, a fatura poderá vir alta!

## 4. *Data Ingestion* e *Analytics*

Para atender todas as atribuições do *checklist*, nós iremos cumprir uma sequência de passos intermediários:

1. Converter o arquivo pdf contendo os endereços dos clientes (tabela do Excel impressa e escaneada) para uma estrutura de DataFrame, utilizando o Document AI (Form Parser).
2. Ler o arquivo csv contendo os dados pessoais dos clientes para uma estrutura de DataFrame, utilizando o Pandas.
3. Verificar a integridade dos dados e se os tipos são adequados a cada coluna.
4. Combinar os dois DataFrames em uma única variável, usando a coluna `id_cliente` como referência.
5. Obter as coordenadas geográficas de cada entrada do *dataset* unificado, por meio do Google Maps.
6. Gerar um novo arquivo `dados_completos.csv` e deixá-lo salvo no diretório do projeto, no Google Drive.
7. Obter a distância e tempo de deslocamento de cada cliente até o ponto marcante "Estádio da Fonte Luminosa, Araraquara - SP".
8. Plotar gráfico da residencia do cliente em relação ao evento.


### 4.1 Converter *pdf* em *DataFrame*

In [69]:
"""
Fazer uma requisição para processamento ao Document AI
"""

# Troque pelos seus dados
project_id = 'Coloque o id do seu projeto no GCP'  # id do projeto 
location = 'local' # ex. us
processor_id = 'coloque o processador no form parser' # O processador Form Parser
file_path = '/content/drive/MyDrive/dados_empresa/enderecos.pdf'
mime_type = 'application/pdf' 


def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    Processes a document using the Document AI Online Processing API.
    """

    # Instantiates a client
    docai_client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    resource_name = docai_client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as file:
        file_content = file.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(content=file_content, mime_type=mime_type)

    # Configure the process request
    request = documentai.ProcessRequest(name=resource_name, raw_document=raw_document)

    # Use the Document AI client to process the sample form
    result = docai_client.process_document(request=request)

    return result.document


def get_table_data(
    rows: Sequence[documentai.Document.Page.Table.TableRow], text: str
) -> List[List[str]]:
    """
    Get Text data from table rows
    """
    all_values: List[List[str]] = []
    for row in rows:
        current_row_values: List[str] = []
        for cell in row.cells:
            current_row_values.append(
                text_anchor_to_text(cell.layout.text_anchor, text)
            )
        all_values.append(current_row_values)
    return all_values


def text_anchor_to_text(text_anchor: documentai.Document.TextAnchor, text: str) -> str:
    """
    Document AI identifies table data by their offsets in the entirity of the
    document's text. This function converts offsets to a string.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in text_anchor.text_segments:
        start_index = int(segment.start_index)
        end_index = int(segment.end_index)
        response += text[start_index:end_index]
    return response.strip().replace("\n", " ")


# Fazer a requisição e enviar o pdf ao processador Form Parser
document = online_process(
    project_id,
    location,
    processor_id,
    file_path,
    mime_type
)

# Extrair cabeçalho e linhas da tabela
header_row_values: List[List[str]] = []
body_row_values: List[List[str]] = []

# caso houvesse mais páginas no documento pdf
for page in document.pages:
    for table in page.tables:
        header_row_values = get_table_data(table.header_rows, document.text)
        body_row_values = get_table_data(table.body_rows, document.text)

# criar um Pandas Dataframe para armazenar os valores em formato tabular
df_enderecos = pd.DataFrame(
    data=body_row_values,
    columns=header_row_values[0],
)

In [70]:
df_enderecos.head()

,id_cliente,logradouro,numero bairro,cep,cidade,uf
0,1,Rua Bento de Miranda Mello,356 Residencial Olivio Benassi,15993-037,Matão,SP
1,2,Rua Ricardo Arruda Ribeiro,745 Água Branca,13425-100,Piracicaba,SP
2,3,Rua Dona Sylvia Finco Costa,870 Jardim Tapajós,12945-080,Atibaia,SP
3,4,Rua Axicará,Vila Isolina Mazzei 433,02081-080,São Paulo,SP
4,5,Rua Diamantino Armando,Vila Esperança 245,18703-800,Avaré,SP


Utilizando a documentação do documentAI, foi realizada a trasformação de um documento escaneado no formato pdf para o formato dataframe. Essa ferramenta pode aumentar significativamente a produtividade na hora de organizar os dados.

### 4.2 Carregar o outro arquivo *csv* em *DataFrame*

In [71]:
df_dados = pd.read_csv('/content/drive/MyDrive/dados_empresa/clientes_dados.csv', sep = ';')

In [72]:
df_dados.head()

,id_cliente,nome,email,sexo,cpf,data_nasc
0,1,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,06/01/1984
1,2,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,06/01/1943
2,3,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,04/02/1970
3,4,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,22/06/1969
4,5,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,07/09/1948


### 4.3 Tratamento dos dados


In [73]:
# Verificando o tipo dos dados do Dataset para a variável df_dados

for i in dict(df_dados.dtypes).items():
  variavel = i[0]
  caracteres = '_'*(30 - len(variavel))
  tipo = i[1]
  print(f'A variável [{variavel}]{caracteres} é do tipo {tipo}')

A variável [id_cliente]____________________ é do tipo int64
A variável [nome]__________________________ é do tipo object
A variável [email]_________________________ é do tipo object
A variável [sexo]__________________________ é do tipo object
A variável [cpf]___________________________ é do tipo object
A variável [data_nasc]_____________________ é do tipo object


Será necessário transformar a data de nascimento para o formato datetime  %d/%m/%y

In [74]:
# Verificando o tipo dos dados do Dataset para a variável df_dados

for i in dict(df_enderecos.dtypes).items():
  variavel = i[0]
  caracteres = '_'*(30 - len(variavel))
  tipo = i[1]
  print(f'A variável [{variavel}]{caracteres} é do tipo {tipo}')

A variável [id_cliente]____________________ é do tipo object
A variável [logradouro]____________________ é do tipo object
A variável [numero bairro]_________________ é do tipo object
A variável [cep]___________________________ é do tipo object
A variável [cidade]________________________ é do tipo object
A variável [uf]____________________________ é do tipo object


Para melhorar a visualização, farei a junção dos dois datasets os quais são ligados pelo `id_cliente` através do método merge.

Para isso as duas colunas `id_cliente` precisam ter o mesmo tipo de variável nos dois datasets.

Será preciso transformar uma das colunas para conseguir juntar os dataframes.

#### 4.3.1 Transformando a coluna id_cliente


In [75]:
# Transformando a coluna id_cliente para o tipo inteiro
df_enderecos['id_cliente'] = df_enderecos['id_cliente'].astype('int64')

#### 4.3.2 Transformando a coluna data_nasc

In [76]:
# Transformando a data de nascimento para o formato datetime
df_dados['data_nasc'] = pd.to_datetime(df_dados['data_nasc'], format = '%d/%m/%Y')

#### 4.3.3 Verificando o tratamento realizado

In [77]:
# Verificando o tipo dos dados do Dataset para a variável df_dados

for i in dict(df_dados.dtypes).items():
  variavel = i[0]
  caracteres = '_'*(30 - len(variavel))
  tipo = i[1]
  print(f'A variável [{variavel}]{caracteres} é do tipo {tipo}')

A variável [id_cliente]____________________ é do tipo int64
A variável [nome]__________________________ é do tipo object
A variável [email]_________________________ é do tipo object
A variável [sexo]__________________________ é do tipo object
A variável [cpf]___________________________ é do tipo object
A variável [data_nasc]_____________________ é do tipo datetime64[ns]


In [78]:
# Verificando o tipo dos dados do Dataset para a variável df_dados

for i in dict(df_enderecos.dtypes).items():
  variavel = i[0]
  caracteres = '_'*(30 - len(variavel))
  tipo = i[1]
  print(f'A variável [{variavel}]{caracteres} é do tipo {tipo}')

A variável [id_cliente]____________________ é do tipo int64
A variável [logradouro]____________________ é do tipo object
A variável [numero bairro]_________________ é do tipo object
A variável [cep]___________________________ é do tipo object
A variável [cidade]________________________ é do tipo object
A variável [uf]____________________________ é do tipo object


Agora podemos realizar a junção correta do dataframe, essa junção servirá para melhorar a capacidade de visualzar os dados.

#### 4.3.4 Combinando em apenas um dataframe

In [79]:
# Combinando datasets 
df_completo = pd.merge(df_enderecos, df_dados, on = "id_cliente")

In [80]:
df_completo.head()

,id_cliente,logradouro,numero bairro,cep,cidade,uf,nome,email,sexo,cpf,data_nasc
0,1,Rua Bento de Miranda Mello,356 Residencial Olivio Benassi,15993-037,Matão,SP,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06
1,2,Rua Ricardo Arruda Ribeiro,745 Água Branca,13425-100,Piracicaba,SP,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,1943-01-06
2,3,Rua Dona Sylvia Finco Costa,870 Jardim Tapajós,12945-080,Atibaia,SP,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,1970-02-04
3,4,Rua Axicará,Vila Isolina Mazzei 433,02081-080,São Paulo,SP,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,1969-06-22
4,5,Rua Diamantino Armando,Vila Esperança 245,18703-800,Avaré,SP,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,1948-09-07


In [81]:
# Ajeitando a variável id_cliente em uma só
df_completo.set_index('id_cliente', inplace = True)

In [82]:
# Verificando o Dataset novamente
df_completo.head()

,logradouro,numero bairro,cep,cidade,uf,nome,email,sexo,cpf,data_nasc
id_cliente,,,,,,,,,,
1,Rua Bento de Miranda Mello,356 Residencial Olivio Benassi,15993-037,Matão,SP,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06
2,Rua Ricardo Arruda Ribeiro,745 Água Branca,13425-100,Piracicaba,SP,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,1943-01-06
3,Rua Dona Sylvia Finco Costa,870 Jardim Tapajós,12945-080,Atibaia,SP,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,1970-02-04
4,Rua Axicará,Vila Isolina Mazzei 433,02081-080,São Paulo,SP,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,1969-06-22
5,Rua Diamantino Armando,Vila Esperança 245,18703-800,Avaré,SP,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,1948-09-07


### 4.4. Obtendo as coordenadas geográficas pela API do Google Maps

In [83]:
# Acessando a API do google maps
gmaps = googlemaps.Client(key = 'coloque sua chave API - Google Maps')

In [85]:
# Gerando uma coluna com endereços completos
enderecos_completos = df_completo['logradouro'] + " , " + df_completo['numero bairro'] + " - " + df_completo['cidade'] + " , " + df_completo['uf']


In [86]:
# Exemplo de endereço completo
enderecos_completos.iloc[0]

'Rua Bento de Miranda Mello , 356 Residencial Olivio Benassi - Matão , SP'

In [87]:
# Obter coordenadas de maneira iterativa
coordenadas = []

# percorrer entradas individuais e fazer requisições individuais
for i, j in enumerate(enderecos_completos):
    geocode_obj = gmaps.geocode(j)

    # extrair apenas lat/lon
    lat = geocode_obj[0]['geometry']['location']['lat']
    lon = geocode_obj[0]['geometry']['location']['lng']

    # adicionar lat/lon à lista
    coordenadas.append([lat, lon])

In [88]:
# Obtendo as coordenadas de maneira iterativa
coordenadas = []

# Percorrendo as entradas de forma individual, fazendo a requisição de forma individual
for i, j in enumerate(enderecos_completos):
  geocode_obj = gmaps.geocode(j)

  # Extraindo latitude e longitude 
  lat = geocode_obj[0]['geometry']['location']['lat']
  lon = geocode_obj[0]['geometry']['location']['lng']

  # Adicionando latitude e longitude a lista de coordenadas
  coordenadas.append([lat, lon])
  

In [89]:
# Verificando a latitude e longitude
geocode_obj[0]['geometry']['location']

{'lat': -23.6826179, 'lng': -46.3587757}

In [90]:
# converter para DataFrame
df_coordenadas = pd.DataFrame(coordenadas, columns=['lat', 'lon'])

# Obter mesmo índice que o dataframe completo
df_coordenadas.set_index(df_coordenadas.index + 1, inplace=True)

### 4.5. Exportar csv com os dados completos

In [91]:
# Combinar coordenadas encontradas os dados completos, em relação ao axis=1
df_completo = pd.concat([df_completo, df_coordenadas], axis=1)

In [94]:
df_completo.head(1)

,logradouro,numero bairro,cep,cidade,uf,nome,email,sexo,cpf,data_nasc,lat,lon
1,Rua Bento de Miranda Mello,356 Residencial Olivio Benassi,15993-037,Matão,SP,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06,-21.61915,-48.368234


In [221]:
# Verificando o tipo dos dados do Dataset para a variável df_dados

for i in dict(df_completo.dtypes).items():
  variavel = i[0]
  caracteres = '_'*(30 - len(variavel))
  tipo = i[1]
  print(f'A variável [{variavel}]{caracteres} é do tipo {tipo}')

A variável [logradouro]____________________ é do tipo object
A variável [numero bairro]_________________ é do tipo object
A variável [cep]___________________________ é do tipo object
A variável [cidade]________________________ é do tipo object
A variável [uf]____________________________ é do tipo object
A variável [nome]__________________________ é do tipo object
A variável [email]_________________________ é do tipo object
A variável [sexo]__________________________ é do tipo object
A variável [cpf]___________________________ é do tipo object
A variável [data_nasc]_____________________ é do tipo datetime64[ns]
A variável [lat]___________________________ é do tipo float64
A variável [lon]___________________________ é do tipo float64
A variável [idade]_________________________ é do tipo int64
A variável [geometry]______________________ é do tipo geometry


As coordenadas foram adicionadas junto aos seus respectivos endereços.

In [92]:
# exportar para arquivo csv dentro do Google Drive (salvando no drive)
df_completo.to_csv('/content/drive/MyDrive/dados_empresa/dados_completos.csv')

### 4.6 Calculando a distância até o evento

In [95]:
local_do_evento = "Estádio da Fonte Luminosa, Araraquara - SP"

In [96]:
# Obter informações sobre o local do evento
gmaps.geocode(local_do_evento)

[{'address_components': [{'long_name': 's/n',
    'short_name': 's/n',
    'types': ['street_number']},
   {'long_name': 'Rua Mauro Pinheiro',
    'short_name': 'R. Mauro Pinheiro',
    'types': ['route']},
   {'long_name': 'Jardim Primavera',
    'short_name': 'Jardim Primavera',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Araraquara',
    'short_name': 'Araraquara',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'São Paulo',
    'short_name': 'SP',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Brazil',
    'short_name': 'BR',
    'types': ['country', 'political']},
   {'long_name': '14802-355',
    'short_name': '14802-355',
    'types': ['postal_code']}],
  'formatted_address': 'R. Mauro Pinheiro, s/n - Jardim Primavera, Araraquara - SP, 14802-355, Brazil',
  'geometry': {'location': {'lat': -21.7752895, 'lng': -48.1702765},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast

In [102]:
# Calculando a distância/tempo entre postos geográficos, com uma data específica
dia_do_evento = datetime(2023, 12, 15)
resultado_distancia = gmaps.directions("-21.620162, -48.366012",
                                       local_do_evento,
                                       mode = "driving",
                                       departure_time = dia_do_evento)

In [103]:
# Exemplo de distância
print(f"A distância deste cliente para o local do evento é de aproximadamente {int(resultado_distancia[0]['legs'][0]['distance']['value'] / 1000)} km")

A distância deste cliente para o local do evento é de aproximadamente 32 km


In [114]:
# Exemplo de tempo em minutos
print(f" O tempo gasto por este cliente para chegar ao evento é de aproximadamente:{round(resultado_distancia[0]['legs'][0]['duration']['value'] /60, 0)} minutos")


 O tempo gasto por este cliente para chegar ao evento é de aproximadamente:35.0 minutos


In [133]:
# obter todas as distâncias e tempos de estrada
lista_distancias = []
dia_do_evento = datetime(2023, 12, 2)

# percorrer entradas individuais e fazer requisições individuais
for i, j in enumerate(enderecos_completos):
    directions_result = gmaps.directions(j,
                                     local_do_evento,
                                     mode="driving",
                                     departure_time=dia_do_evento)

    # adicionar às listas
    distancia = directions_result[0]['legs'][0]['distance']['value']
    tempo = directions_result[0]['legs'][0]['duration']['value'] / 60
    lista_distancias.append([distancia, tempo])

In [124]:
# Obtendo todas as distâncias e e tempo na estrada

lista_distancia = []
dia_do_evento = datetime(2023, 12, 3)

# Percorrendo entradas individuais e fazendo a requisição individual de cada cliente
for i, j in enumerate(enderecos_completos):
  resultado_distancia = gmaps.directions(j,
                                         local_do_evento,
                                         mode = 'driving',
                                         departure_time = dia_do_evento)
  
  # Adicionando o resultado na lista vazia
  distancia = round(resultado_distancia[0]['legs'][0]['distance']['value'] / 1000, 0)
  tempo = round(resultado_distancia[0]['legs'][0]['duration']['value'] / 60, 0)
  lista_distancia.append([distancia, tempo])


In [125]:
# Convertendo a lista para o formato Dataframe
df_distancias = pd.DataFrame(lista_distancia, columns = ['distancia', 'tempo'])

# Colocando ao mesmo índice do dataframe original
df_distancias.set_index(df_distancias.index + 1, inplace = True)


In [126]:
# Combinando coordenadas aos dados existentes, em relação ao axis = 1
df_evento = pd.concat([df_completo['nome'], df_distancias], axis = 1)

In [132]:
df_evento.head()

,nome,distancia,tempo
1,Sebastiana Tânia Mendes,33.0,36.0
2,Andrea Débora Campos,145.0,116.0
3,Rosa Jéssica Isis Barros,254.0,186.0
4,Vinicius Carlos Eduardo da Rocha,279.0,208.0
5,Cecília Milena Elisa Vieira,185.0,156.0


### 4.7. Calculando a idade média dos clientes


In [141]:
from dateutil.relativedelta import relativedelta

# Calcule a idade
hoje = datetime.today()
df_completo['idade'] = df_completo['data_nasc'].apply(lambda x: relativedelta(hoje, x).years)

df_completo.head()


,logradouro,numero bairro,cep,cidade,uf,nome,email,sexo,cpf,data_nasc,lat,lon,idade
1,Rua Bento de Miranda Mello,356 Residencial Olivio Benassi,15993-037,Matão,SP,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06,-21.619150,-48.368234,39
2,Rua Ricardo Arruda Ribeiro,745 Água Branca,13425-100,Piracicaba,SP,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,1943-01-06,-22.748635,-47.630113,80
3,Rua Dona Sylvia Finco Costa,870 Jardim Tapajós,12945-080,Atibaia,SP,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,1970-02-04,-23.111416,-46.536782,53
4,Rua Axicará,Vila Isolina Mazzei 433,02081-080,São Paulo,SP,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,1969-06-22,-23.495099,-46.602611,53
5,Rua Diamantino Armando,Vila Esperança 245,18703-800,Avaré,SP,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,1948-09-07,-23.122264,-48.920483,74


In [146]:
print(f"A idade média dos clientes é de aproximadamente: {round(df_completo['idade'].mean(), 0)} anos")

A idade média dos clientes é de aproximadamente: 56.0 anos


### 4.8 Identificando o cliente que mora mais longe do local

In [158]:
# Calculando o cliente com a maior distância 
distancia_max = df_evento['distancia'].max()

# Buscando atravéz do índice o cliente
indice_max = df_evento['distancia'].idxmax()

# Organizando as informações do cliente
linha_max = df_evento.loc[indice_max]

print("O maior valor na coluna é:", distancia_max)
print("O Cliente mais distânte chama-se:")
print(linha_max)


O maior valor na coluna é: 371.0
O Cliente mais distânte chama-se:
nome         Renato Vicente Antonio Freitas
distancia                             371.0
tempo                                 281.0
Name: 15, dtype: object


### 4.9 Plotando um mapa da localização dos clientes em relação ao evento

In [177]:
!pip install geopandas -q
!pip install folium -q


In [252]:
# Importando os pacotes necessários
import geopandas as gpd
import folium
from folium.map import Icon
from folium import plugins
from shapely.geometry import Point

In [253]:
# Separando as coordenadas do evento
coordenadas = []

for lat, lon in zip(df_completo.lat, df_completo.lon):
  coordenadas.append([lat, lon])


### 4.10 Plotando um mapa de ícones que marcam a região do cliente

In [266]:
# Ajustando o mapa
mapa_ponto = folium.Map(location = [-21.7752011,-48.1708143],
                  zoom_start = 20)

# Adicionando um marcador no local do evento
folium.Marker(
    [-21.7752011,-48.1708143],
    popup = "<i>Estádio da Fonte Luminosa, Araraquara - SP</i>",
    tooltip = "Clique Aqui!",
    icon = folium.Icon(color = 'green'),
    
).add_to(mapa_ponto)

# Plotando o mapa
mapa_ponto.add_child(plugins.MarkerCluster(coordenadas))
mapa_ponto

### 4.11 Plotando um mapa de calor da região dos clientes

In [268]:
# Ajustando o mapa de calor
mapa_calor = folium.Map(location = [-21.7752011,-48.1708143],
                  zoom_start = 20)

# Adicionando um marcador no local do evento
folium.Marker(
    [-21.7752011,-48.1708143],
    popup = "<i>Estádio da Fonte Luminosa, Araraquara - SP</i>",
    tooltip = "Clique Aqui!",
    icon = folium.Icon(color = 'green'),
    
).add_to(mapa_calor)

# Plotando o mapa de calor
mapa_calor.add_child(plugins.HeatMap(coordenadas))
mapa_calor

## 5. Considerações finais

Business Intelligence (BI) é importante para as empresas por diversas razões. Em resumo, o BI é um conjunto de técnicas e ferramentas que ajudam as empresas a coletar, analisar e transformar dados em informações úteis para tomar decisões de negócios mais informadas e estratégicas.

Em resumo, o BI é importante para as empresas porque ajuda a transformar dados em informações úteis que podem levar a decisões mais informadas, eficiência operacional e vantagem competitiva.

Neste projeto transformamos dados coletados de forma irregular e transformamos em BI, utilizando as ferramentas da Google Cloud Platform. Por fim, utlizamos as ferramentas de visualização de mapas da biblioteca Geopandas para verificar a distância dos clientes até o evento em questão.

Para segurança, retirei as chaves de API da minha conta GCP, Caso queira replicar será necessário criar chaves de API para o documentAI e e Google Maps.